In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import sklearn
from math import sqrt

In [4]:
# import historical data 
from help_functions import get_energy_data

consumption_data = get_energy_data.get_data(
    'hourly_ec_20201107_20231107.csv')

# add weekdays, months, hours seperately 
# monday = 0, sunday = 6
consumption_data['weekday'] = consumption_data.index.weekday
consumption_data['hour'] = consumption_data.index.hour
consumption_data['month'] = consumption_data.index.month

,date,beginning,energy_consumption,weekday,hour,month
date_time,,,,,,
2020-11-07 00:00:00,2020-11-07,00:00:00,46.97475,5,0,11
2020-11-07 01:00:00,2020-11-07,01:00:00,45.35550,5,1,11
2020-11-07 02:00:00,2020-11-07,02:00:00,44.66450,5,2,11
2020-11-07 03:00:00,2020-11-07,03:00:00,44.76300,5,3,11
2020-11-07 04:00:00,2020-11-07,04:00:00,44.81150,5,4,11
...,...,...,...,...,...,...
2023-11-07 06:00:00,2023-11-07,06:00:00,59.43725,1,6,11
2023-11-07 07:00:00,2023-11-07,07:00:00,63.89000,1,7,11
2023-11-07 08:00:00,2023-11-07,08:00:00,66.64725,1,8,11


In [28]:
# Create dummy variables for month and hour to represent the seasonal components
#consumption_data = pd.get_dummies(
#    consumption_data, columns=['month', 'hour'], prefix=['month', 'hour'], dtype=int)

# Drop 'smoothed', 'date', 'beginning', 'end' columns
#consumption_data = consumption_data.drop(columns=['date', 'beginning'])
# Define the dependent variable (target) and independent variables (features)
y_mod1 = consumption_data['energy_consumption']
X_mod1 = consumption_data.drop(columns=['energy_consumption', 'weekday'])

# Add a constant for the intercept term
X_mod1 = sm.add_constant(X_mod1)

X_mod1
# Fit the seasonal linear regression model
mod1 = sm.OLS(y_mod1, X_mod1).fit()
mod1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     energy_consumption   R-squared:                       0.579
Model:                            OLS   Adj. R-squared:                  0.578
Method:                 Least Squares   F-statistic:                     1060.
Date:                Wed, 08 Nov 2023   Prob (F-statistic):               0.00
Time:                        14:01:57   Log-Likelihood:                -85596.
No. Observations:               26291   AIC:                         1.713e+05
Df Residuals:                   26256   BIC:                         1.715e+05
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.711e+12   5.91e+12      0.458      0.647   -8.88e+12    1.43e+13
month_1    -1.905e+12   3.23e+12     -0.589      0.556   -8.24e+12    4.43e+12
month_2    -1.905e+12   3.23e+12     -0.589      0.556   -8.24e+12    4.43e+12
month_3    -1.905e+12   3.23e+12     -0.589      0.556   -8.24e+12    4.43e+12
month_4    -1.905e+12   3.23e+12     -0.589      0.556   -8.24e+12    4.43e+12
month_5    -1.905e+12   3.23e+12     -0.589      0.556   -8.24e+12    4.43e+12
month_6    -1.905e+12   3.23e+12     -0.589      0.556   -8.24e+12    4.43e+12
month_7    -1.905e+12   3.23e+12     -0.589      0.556   -8.24e+12    4.43e+12
month_8    -1.905e+12   3.23e+12     -0.589      0.556   -8.24e+12    4.43e+12
month_9    -1.905e+12   3.23e+12     -0.589      0.556   -8.24e+12    4.43e+12
month_10   -1.905e+12   3.23e+12     -0.589      0.556   -8.24e+12    4.43e+12
month_11   -1.905e+12   3.23e+12     -0.589      0.556   -8.24e+12    4.43e+12
month_12   -1.905e+12   3.23e+12     -0.589      0.556   -8.24e+12    4.43e+12
hour_0      -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_1      -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_2      -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_3      -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_4      -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_5      -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_6      -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_7      -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_8      -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_9      -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_10     -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_11     -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_12     -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_13     -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_14     -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_15     -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_16     -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_17     -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_18     -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_19     -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_20     -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_21     -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
hour_22     -8.06e+11   6.31e+12     -0.128      0.898   -1.32e+13    1.16e+13
ho

In [31]:
# create new dataframe containing future date_times and indepentent variables
last_ts = consumption_data.index[-1]
horizon = pd.date_range(start=last_ts + pd.DateOffset(
    hours=1), periods=200, freq='H')

energy_forecast1 = pd.DataFrame({'date_time': horizon})
energy_forecast1.set_index('date_time', inplace=True)
# monday = 0, sunday = 6
energy_forecast1['weekday'] = energy_forecast1.index.weekday
energy_forecast1['hour'] = energy_forecast1.index.hour
energy_forecast1['month'] = energy_forecast1.index.month

energy_forecast1 = pd.get_dummies(
    energy_forecast1, columns=['month', 'hour'], prefix=['month', 'hour'], dtype=int)

# add dummies of months not contained in energy_forecast
missing_columns = set(consumption_data.columns) - set(energy_forecast1.columns)
for col in missing_columns:
    energy_forecast1[col] = 0

# Point forecasts
X_fc1 = energy_forecast1.drop(columns=['weekday'])
X_fc1 = sm.add_constant(X_fc1)

# Make predictions
predictions = mod1.predict(X_fc1)
predictions


date_time
2023-11-07 11:00:00    8.060364e+11
2023-11-07 12:00:00    1.904805e+12
2023-11-07 13:00:00    1.904805e+12
2023-11-07 14:00:00    1.904805e+12
2023-11-07 15:00:00    1.904805e+12
                           ...     
2023-11-15 14:00:00    1.904805e+12
2023-11-15 15:00:00    1.904805e+12
2023-11-15 16:00:00    1.904805e+12
2023-11-15 17:00:00    1.904805e+12
2023-11-15 18:00:00    1.904805e+12
Length: 200, dtype: float64

### Import historical energy consumption data and add required columns for model 2 

In [3]:
from help_functions import get_energy_data

energyconsumption_data = get_energy_data.get_data(
    'hourly_ec_20201107_20231107.csv')

# add weekdays, months, hours seperately
# monday = 0, sunday = 6
energyconsumption_data['weekday'] = energyconsumption_data.index.weekday
energyconsumption_data['hour'] = energyconsumption_data.index.hour
energyconsumption_data['month'] = energyconsumption_data.index.month


# Define the mapping of months to seasons
season_mapping = {
    'summer': [10, 11, 12, 1, 2, 3],
    'winter': [4, 5, 6, 7, 8, 9],
}

# create season dummy variable
for season, months in season_mapping.items():
    energyconsumption_data[season] = energyconsumption_data['month'].apply(
        lambda x: 1 if x in months else 0)


# Define mapping of hours to timeframes (based on graph)
time_mapping = {
    'low_consumption_time': list(range(7)),  # differs a lot weekend/weekday
    'high_consumption_time': list(range(7, 20)),
    'transition_time': [6, 20, 21, 22, 23]}

# create timeframe dummy variable
for timeframe, hours in time_mapping.items():
    energyconsumption_data[timeframe] = energyconsumption_data['hour'].apply(
        lambda x: 1 if x in hours else 0)

# Define mapping of days to weekend or not (based on graph)
day_mapping = {
    'working_day': list(range(5)),  # differs a lot weekend/weekday
    'weekend_day': [5,6]}

# create timeframe dummy variable
for daymapping, day in time_mapping.items():
    energyconsumption_data[daymapping] = energyconsumption_data['weekday'].apply(
        lambda x: 1 if x in day else 0)

energyconsumption_data

,date,beginning,energy_consumption,weekday,hour,month,summer,winter,low_consumption_time,high_consumption_time,transition_time
date_time,,,,,,,,,,,
2020-11-07 00:00:00,2020-11-07,00:00:00,46.97475,5,0,11,1,0,1,0,0
2020-11-07 01:00:00,2020-11-07,01:00:00,45.35550,5,1,11,1,0,1,0,0
2020-11-07 02:00:00,2020-11-07,02:00:00,44.66450,5,2,11,1,0,1,0,0
2020-11-07 03:00:00,2020-11-07,03:00:00,44.76300,5,3,11,1,0,1,0,0
2020-11-07 04:00:00,2020-11-07,04:00:00,44.81150,5,4,11,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2023-11-07 06:00:00,2023-11-07,06:00:00,59.43725,1,6,11,1,0,1,0,0
2023-11-07 07:00:00,2023-11-07,07:00:00,63.89000,1,7,11,1,0,1,0,0
2023-11-07 08:00:00,2023-11-07,08:00:00,66.64725,1,8,11,1,0,1,0,0


### Fit model 

In [5]:
y_ec = energyconsumption_data['energy_consumption']
X_ec = energyconsumption_data.drop(columns=['energy_consumption', 'date', 
                                            'beginning', 'weekday', 'hour', 'month', 'summer',  
                                            'transition_time'])

 
# add constant for the intercept term
X_ec = sm.add_constant(X_ec)

X_ec
# fit seasonal linear regression model
model_ec1 = sm.OLS(y_ec, X_ec).fit()
model_ec1.summary()

NameError: name 'sm' is not defined

In [1]:
# create new dataframe containing future date_times and indepentent variables

last_ts = energyconsumption_data.index[-1]
horizon = pd.date_range(start=last_ts + pd.DateOffset(
    hours=1), periods=200, freq='H')

energy_forecast_ec = pd.DataFrame({'date_time': horizon})
energy_forecast_ec.set_index('date_time', inplace=True)

# monday = 0, sunday = 6
energy_forecast_ec['weekday'] = energy_forecast_ec.index.weekday
energy_forecast_ec['hour'] = energy_forecast_ec.index.hour
energy_forecast_ec['month'] = energy_forecast_ec.index.month

season_mapping = {
    'winter': [12, 1, 2],
    'spring': [3, 4, 5],
    'summer': [6, 7, 8],
    'autumn': [9, 10, 11]
}

# create season dummy variable
for season, month in season_mapping.items():
    energy_forecast_ec[season] = np.where(
        energy_forecast_ec['month'].isin(month), 1, 0)

# Define mapping of hours to timeframes (based on graph)
time_mapping = {
    'low_consumption_time': list(range(7)),  # differs a lot weekend/weekday
    'high_consumption_time': list(range(7, 20)),
    'transition_time': [6, 20, 21, 22, 23]}

# Create timeframe dummy variables
for timeframe, hours in time_mapping.items():
    energy_forecast_ec[timeframe] = np.where(
        energy_forecast_ec['hour'].isin(hours), 1, 0)

# Point forecasts
X_fc_ec = energy_forecast_ec.drop(columns=['weekday', 'hour', 'month',
                                           'spring', 'autumn','summer', 'transition_time'])
X_fc_ec = sm.add_constant(X_fc_ec)

X_fc_ec
# Make predictions
#predictions_ec = model_ec1.predict(X_fc_ec)
#predictions_ec.head(20)

NameError: name 'energyconsumption_data' is not defined